In [ ]:
# Configurações Iniciais e Imports
import os
import sys
import logging
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import trimesh
from pathlib import Path

# Adicionar o diretório raiz do projeto ao path
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.utils.metrics import run_evaluation_on_dataset, load_landmarks_from_json
from src.utils.helpers import setup_logging, list_stl_files, save_landmarks_to_json
from src.core.landmarks import LANDMARK_NAMES

# Configurar logging
setup_logging(log_level=logging.INFO)

# Configurar estilo dos gráficos
plt.style.use('default')
sns.set_palette("husl")

print(f"Diretório de trabalho: {os.getcwd()}")
print(f"Path do módulo: {module_path}")
print(f"Landmarks definidos: {len(LANDMARK_NAMES)}")

In [ ]:
# Configurar diretórios
BASE_DIR = module_path
RESULTS_DIR = os.path.join(BASE_DIR, "results")
RESULTS_GEOM_DIR = os.path.join(RESULTS_DIR, "geometric") 
RESULTS_ML_DIR = os.path.join(RESULTS_DIR, "ml")
GROUND_TRUTH_DIR = os.path.join(BASE_DIR, "data", "ground_truth")
DATA_DIR = os.path.join(BASE_DIR, "data", "skulls")

# Criar diretórios se não existirem
for directory in [RESULTS_GEOM_DIR, RESULTS_ML_DIR, GROUND_TRUTH_DIR, DATA_DIR]:
    os.makedirs(directory, exist_ok=True)

print(f"Diretórios configurados:")
print(f"  Resultados gerais: {RESULTS_DIR}")
print(f"  Resultados geométricos: {RESULTS_GEOM_DIR}")
print(f"  Resultados ML: {RESULTS_ML_DIR}")
print(f"  Ground truth: {GROUND_TRUTH_DIR}")
print(f"  Dados originais: {DATA_DIR}")

In [ ]:
# Criar dados dummy para demonstração
# Em um cenário real, estes dados viriam de execuções do main.py

print("=== Criando Dados Dummy para Demonstração ===")
print("⚠️  Em um cenário real, estes dados seriam gerados pelo main.py")

DUMMY_FILE_IDS = ["dummy_A", "dummy_B", "dummy_C"]

for file_id in DUMMY_FILE_IDS:
    # 1. Criar arquivo STL dummy (se não existir)
    dummy_stl_path = os.path.join(DATA_DIR, f"{file_id}.stl")
    if not os.path.exists(dummy_stl_path):
        mesh_dummy = trimesh.primitives.Sphere(radius=50 + np.random.randint(10))
        mesh_dummy.export(dummy_stl_path)
        
    # 2. Criar Ground Truth dummy com coordenadas realistas
    gt_path = os.path.join(GROUND_TRUTH_DIR, f"{file_id}_landmarks_gt.json")
    if not os.path.exists(gt_path):
        # Gerar coordenadas GT realistas baseadas em anatomia
        base_coords = {
            "Glabela": [0, 50, 50],
            "Nasion": [0, 45, 40], 
            "Bregma": [0, 0, 100],
            "Opisthocranion": [0, -50, 50],
            "Euryon_Esquerdo": [-50, 0, 50],
            "Euryon_Direito": [50, 0, 50],
            "Vertex": [0, 5, 100],
            "Inion": [0, -45, 35]
        }
        
        # Adicionar variação individual
        gt_data = {}
        for name, base_coord in base_coords.items():
            # Adicionar variação de ±5mm em cada eixo
            variation = (np.random.rand(3) - 0.5) * 10
            gt_data[name] = (np.array(base_coord) + variation).tolist()
        
        # Simular alguns landmarks ausentes no GT
        if file_id == "dummy_C":
            gt_data["Vertex"] = None
            
        with open(gt_path, "w") as f:
            json.dump(gt_data, f, indent=4)
    
    # 3. Criar resultados geométricos dummy (com erro simulado)
    geom_pred_path = os.path.join(RESULTS_GEOM_DIR, f"{file_id}_landmarks.json")
    if not os.path.exists(geom_pred_path):
        gt_data = load_landmarks_from_json(gt_path)
        if gt_data:
            geom_pred = {}
            for name, gt_coord in gt_data.items():
                if gt_coord is not None and np.random.rand() > 0.15:  # 85% detecção
                    # Erro simulado: ±3mm com viés
                    error = (np.random.rand(3) - 0.5) * 6
                    geom_pred[name] = (np.array(gt_coord) + error).tolist()
                else:
                    geom_pred[name] = None
            
            with open(geom_pred_path, "w") as f:
                json.dump(geom_pred, f, indent=4)
    
    # 4. Criar resultados ML dummy (erro menor, melhor taxa)
    ml_pred_path = os.path.join(RESULTS_ML_DIR, f"{file_id}_landmarks.json")
    if not os.path.exists(ml_pred_path):
        gt_data = load_landmarks_from_json(gt_path)
        if gt_data:
            ml_pred = {}
            for name, gt_coord in gt_data.items():
                if gt_coord is not None and np.random.rand() > 0.05:  # 95% detecção
                    # Erro menor: ±1.5mm
                    error = (np.random.rand(3) - 0.5) * 3
                    ml_pred[name] = (np.array(gt_coord) + error).tolist()
                else:
                    ml_pred[name] = None
            
            with open(ml_pred_path, "w") as f:
                json.dump(ml_pred, f, indent=4)

print(f"✅ Dados dummy criados para {len(DUMMY_FILE_IDS)} arquivos")
print(f"   Ground truth: {len(DUMMY_FILE_IDS)} arquivos")
print(f"   Resultados geométricos: {len(list(Path(RESULTS_GEOM_DIR).glob('*.json')))} arquivos")
print(f"   Resultados ML: {len(list(Path(RESULTS_ML_DIR).glob('*.json')))} arquivos")

In [ ]:
print("=== Executando Avaliação dos Métodos ===")

# Avaliação do Método Geométrico
print("\n🔍 Avaliando Método Geométrico...")
results_geom_df, summary_geom_df = run_evaluation_on_dataset(
    results_dir=RESULTS_GEOM_DIR, 
    ground_truth_dir=GROUND_TRUTH_DIR, 
    method_name="Geometric"
)

if not results_geom_df.empty:
    print(f"✅ Avaliação geométrica concluída: {len(results_geom_df)} registros")
    overall_geom_rate = results_geom_df["Detected"].mean() * 100
    overall_geom_error = results_geom_df["Error"].mean()
    print(f"   Taxa de detecção: {overall_geom_rate:.1f}%")
    print(f"   Erro médio: {overall_geom_error:.3f} mm")
else:
    print("❌ Avaliação geométrica falhou")

# Avaliação do Método ML
print("\n🔍 Avaliando Método Machine Learning...")
results_ml_df, summary_ml_df = run_evaluation_on_dataset(
    results_dir=RESULTS_ML_DIR, 
    ground_truth_dir=GROUND_TRUTH_DIR, 
    method_name="ML"
)

if not results_ml_df.empty:
    print(f"✅ Avaliação ML concluída: {len(results_ml_df)} registros")
    overall_ml_rate = results_ml_df["Detected"].mean() * 100
    overall_ml_error = results_ml_df["Error"].mean()
    print(f"   Taxa de detecção: {overall_ml_rate:.1f}%")
    print(f"   Erro médio: {overall_ml_error:.3f} mm")
else:
    print("❌ Avaliação ML falhou")

# Combinar resultados para análise comparativa
results_combined_df = pd.DataFrame()
summary_combined_df = pd.DataFrame()

if not results_geom_df.empty and not results_ml_df.empty:
    results_combined_df = pd.concat([results_geom_df, results_ml_df], ignore_index=True)
    summary_combined_df = pd.concat([summary_geom_df, summary_ml_df], ignore_index=True)
    print(f"\n📊 Datasets combinados criados:")
    print(f"   Resultados detalhados: {len(results_combined_df)} registros")
    print(f"   Resumo por landmark: {len(summary_combined_df)} registros")
elif not results_geom_df.empty:
    results_combined_df = results_geom_df
    summary_combined_df = summary_geom_df
    print("⚠️  Apenas resultados geométricos disponíveis")
elif not results_ml_df.empty:
    results_combined_df = results_ml_df
    summary_combined_df = summary_ml_df
    print("⚠️  Apenas resultados ML disponíveis")
else:
    print("❌ Nenhum resultado válido para análise")

# Salvar datasets combinados
if not results_combined_df.empty:
    results_csv = os.path.join(RESULTS_DIR, "evaluation_combined_detailed.csv")
    summary_csv = os.path.join(RESULTS_DIR, "evaluation_combined_summary.csv")
    
    results_combined_df.to_csv(results_csv, index=False)
    summary_combined_df.to_csv(summary_csv, index=False)
    
    print(f"\n💾 Resultados salvos:")
    print(f"   Detalhados: {results_csv}")
    print(f"   Resumo: {summary_csv}")

In [ ]:
# Exibir resumo dos resultados
if not results_combined_df.empty:
    print("=== Resumo dos Resultados Detalhados ===")
    print(results_combined_df.head(10).round(3))
    
if not summary_combined_df.empty:
    print("\n=== Resumo Agregado por Landmark ===")
    # Formatar colunas numéricas
    display_summary = summary_combined_df.copy()
    numeric_cols = ['MeanError', 'StdError', 'MedianError', 'MinError', 'MaxError', 'DetectionRate']
    for col in numeric_cols:
        if col in display_summary.columns:
            if col == 'DetectionRate':
                display_summary[col] = display_summary[col].round(1).astype(str) + '%'
            else:
                display_summary[col] = display_summary[col].round(3)
    
    print(display_summary[['Landmark', 'MeanError', 'DetectionRate', 'NumDetected', 'NumGT']].head(10))

In [ ]:
print("=== Gerando Visualizações Comparativas ===")

if not results_combined_df.empty:
    # Preparar dados para visualização (remover NaN para os gráficos)
    plot_data = results_combined_df.dropna(subset=['Error'])
    
    if len(plot_data) > 0:
        # Configurar subplots
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('Análise Comparativa dos Métodos de Detecção', fontsize=16, fontweight='bold')
        
        # Gráfico 1: Boxplot do Erro por Método
        ax1 = axes[0, 0]
        sns.boxplot(data=plot_data, x="Method", y="Error", ax=ax1)
        ax1.set_title("Distribuição de Erros por Método")
        ax1.set_ylabel("Erro de Detecção (mm)")
        ax1.grid(True, alpha=0.3)
        
        # Gráfico 2: Boxplot do Erro por Landmark
        ax2 = axes[0, 1]
        # Selecionar apenas landmarks com dados suficientes
        landmark_counts = plot_data['Landmark'].value_counts()
        popular_landmarks = landmark_counts[landmark_counts >= 2].index[:6]  # Top 6
        
        if len(popular_landmarks) > 0:
            landmark_data = plot_data[plot_data['Landmark'].isin(popular_landmarks)]
            sns.boxplot(data=landmark_data, x="Landmark", y="Error", ax=ax2)
            ax2.set_title("Distribuição de Erros por Landmark")
            ax2.set_ylabel("Erro de Detecção (mm)")
            ax2.tick_params(axis='x', rotation=45)
            ax2.grid(True, alpha=0.3)
        else:
            ax2.text(0.5, 0.5, 'Dados insuficientes\npara landmarks', 
                    ha='center', va='center', transform=ax2.transAxes)
            ax2.set_title("Distribuição por Landmark")
        
        # Gráfico 3: Taxa de Detecção por Método
        ax3 = axes[1, 0]
        if not summary_combined_df.empty:
            method_summary = summary_combined_df.groupby('Method')['DetectionRate'].mean().reset_index()
            method_summary['Method'] = method_summary['Method'].fillna('Unknown')
            
            bars = sns.barplot(data=method_summary, x="Method", y="DetectionRate", ax=ax3)
            ax3.set_title("Taxa de Detecção Média por Método")
            ax3.set_ylabel("Taxa de Detecção (%)")
            ax3.set_ylim(0, 105)
            ax3.grid(True, alpha=0.3)
            
            # Adicionar valores nas barras
            for i, bar in enumerate(bars.patches):
                height = bar.get_height()
                if not np.isnan(height):
                    ax3.text(bar.get_x() + bar.get_width()/2., height + 1,
                           f'{height:.1f}%', ha='center', va='bottom')
        else:
            ax3.text(0.5, 0.5, 'Dados de resumo\nnão disponíveis', 
                    ha='center', va='center', transform=ax3.transAxes)
            ax3.set_title("Taxa de Detecção por Método")
        
        # Gráfico 4: Scatter Error vs Landmark para ambos métodos
        ax4 = axes[1, 1]
        if 'Method' in plot_data.columns and len(plot_data['Method'].unique()) > 1:
            for method in plot_data['Method'].unique():
                method_data = plot_data[plot_data['Method'] == method]
                ax4.scatter(range(len(method_data)), method_data['Error'], 
                          label=method, alpha=0.6, s=30)
            
            ax4.set_title("Erro de Detecção por Amostra")
            ax4.set_xlabel("Índice da Amostra")
            ax4.set_ylabel("Erro (mm)")
            ax4.legend()
            ax4.grid(True, alpha=0.3)
        else:
            ax4.scatter(range(len(plot_data)), plot_data['Error'], alpha=0.6)
            ax4.set_title("Erro de Detecção por Amostra")
            ax4.set_xlabel("Índice da Amostra")
            ax4.set_ylabel("Erro (mm)")
            ax4.grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        # Salvar gráfico
        comparison_plot_path = os.path.join(RESULTS_DIR, "comparison_analysis.png")
        plt.savefig(comparison_plot_path, dpi=300, bbox_inches='tight')
        print(f"✅ Gráfico de análise comparativa salvo em: {comparison_plot_path}")
        
        plt.show()
        
    else:
        print("❌ Não há dados válidos (sem NaN) para gerar gráficos")
else:
    print("❌ Dados combinados não disponíveis para visualização")

In [ ]:
print("=== Análise Estatística Detalhada ===")

if not results_combined_df.empty:
    # Estatísticas gerais por método
    print("\n📊 Estatísticas Gerais por Método:")
    
    for method in results_combined_df['Method'].unique():
        method_data = results_combined_df[results_combined_df['Method'] == method]
        valid_errors = method_data['Error'].dropna()
        
        if len(valid_errors) > 0:
            print(f"\n{method}:")
            print(f"  Total de detecções: {len(method_data)}")
            print(f"  Detecções válidas: {len(valid_errors)} ({len(valid_errors)/len(method_data)*100:.1f}%)")
            print(f"  Erro médio: {valid_errors.mean():.3f} ± {valid_errors.std():.3f} mm")
            print(f"  Erro mediano: {valid_errors.median():.3f} mm")
            print(f"  Erro mínimo: {valid_errors.min():.3f} mm")
            print(f"  Erro máximo: {valid_errors.max():.3f} mm")
            print(f"  Percentil 95: {valid_errors.quantile(0.95):.3f} mm")
        else:
            print(f"\n{method}: Sem erros válidos para análise")
    
    # Análise por landmark
    if not summary_combined_df.empty:
        print("\n🎯 Top Landmarks por Taxa de Detecção:")
        
        # Agrupar por landmark (média entre métodos)
        landmark_avg = summary_combined_df.groupby('Landmark').agg({
            'DetectionRate': 'mean',
            'MeanError': 'mean',
            'NumDetected': 'sum',
            'NumGT': 'sum'
        }).round(3)
        
        # Ordenar por taxa de detecção
        landmark_avg_sorted = landmark_avg.sort_values('DetectionRate', ascending=False)
        
        print(landmark_avg_sorted.head(8))
        
        # Landmarks mais desafiadores
        print("\n⚠️  Landmarks Mais Desafiadores (baixa taxa de detecção):")
        challenging = landmark_avg_sorted.tail(3)
        for landmark, data in challenging.iterrows():
            print(f"  {landmark}: {data['DetectionRate']:.1f}% detecção, erro {data['MeanError']:.3f}mm")
        
        # Landmarks mais precisos
        print("\n✅ Landmarks Mais Precisos (menor erro):")
        precise = landmark_avg.dropna(subset=['MeanError']).sort_values('MeanError').head(3)
        for landmark, data in precise.iterrows():
            print(f"  {landmark}: {data['MeanError']:.3f}mm erro, {data['DetectionRate']:.1f}% detecção")

    # Comparação estatística entre métodos (se ambos disponíveis)
    methods = results_combined_df['Method'].unique()
    if len(methods) >= 2:
        print("\n🔍 Comparação Estatística entre Métodos:")
        
        method1_errors = results_combined_df[results_combined_df['Method'] == methods[0]]['Error'].dropna()
        method2_errors = results_combined_df[results_combined_df['Method'] == methods[1]]['Error'].dropna()
        
        if len(method1_errors) > 0 and len(method2_errors) > 0:
            # Teste t (assumindo normalidade)
            from scipy import stats
            t_stat, p_value = stats.ttest_ind(method1_errors, method2_errors)
            
            print(f"  Diferença de médias: {method1_errors.mean() - method2_errors.mean():.3f} mm")
            print(f"  Teste t: t={t_stat:.3f}, p={p_value:.4f}")
            
            if p_value < 0.05:
                better_method = methods[0] if method1_errors.mean() < method2_errors.mean() else methods[1]
                print(f"  ✅ {better_method} é significativamente melhor (p < 0.05)")
            else:
                print(f"  ⚖️  Não há diferença significativa entre os métodos (p >= 0.05)")
        else:
            print("  Dados insuficientes para comparação estatística")

else:
    print("❌ Dados não disponíveis para análise estatística")

In [ ]:
print("=== Relatório de Performance por Landmark ===")

if not summary_combined_df.empty:
    # Criar relatório por landmark
    landmark_report = []
    
    for landmark in LANDMARK_NAMES:
        landmark_data = summary_combined_df[summary_combined_df['Landmark'] == landmark]
        
        if not landmark_data.empty:
            # Calcular métricas médias entre métodos
            avg_detection_rate = landmark_data['DetectionRate'].mean()
            avg_error = landmark_data['MeanError'].mean()
            total_detected = landmark_data['NumDetected'].sum()
            total_gt = landmark_data['NumGT'].sum()
            
            # Classificar dificuldade
            if avg_detection_rate >= 90:
                difficulty = "Fácil 😊"
            elif avg_detection_rate >= 70:
                difficulty = "Moderado 😐"
            else:
                difficulty = "Difícil 😓"
            
            # Classificar precisão
            if not np.isnan(avg_error):
                if avg_error <= 2.0:
                    precision = "Alta ✅"
                elif avg_error <= 5.0:
                    precision = "Média ⚠️"
                else:
                    precision = "Baixa ❌"
                error_str = f"{avg_error:.2f}mm"
            else:
                precision = "N/A"
                error_str = "N/A"
            
            landmark_report.append({
                'Landmark': landmark,
                'Taxa_Detecção': f"{avg_detection_rate:.1f}%",
                'Erro_Médio': error_str,
                'Dificuldade': difficulty,
                'Precisão': precision,
                'Detecções': f"{total_detected}/{total_gt}"
            })
    
    # Criar DataFrame do relatório
    if landmark_report:
        report_df = pd.DataFrame(landmark_report)
        
        print("\n📋 Relatório Resumido por Landmark:")
        print(report_df.to_string(index=False))
        
        # Salvar relatório
        report_path = os.path.join(RESULTS_DIR, "landmark_performance_report.csv")
        report_df.to_csv(report_path, index=False)
        print(f"\n💾 Relatório salvo em: {report_path}")
        
        # Estatísticas do relatório
        easy_count = sum(1 for item in landmark_report if "Fácil" in item['Dificuldade'])
        moderate_count = sum(1 for item in landmark_report if "Moderado" in item['Dificuldade'])
        hard_count = sum(1 for item in landmark_report if "Difícil" in item['Dificuldade'])
        
        print(f"\n📊 Distribuição de Dificuldade:")
        print(f"   Fáceis: {easy_count} landmarks")
        print(f"   Moderados: {moderate_count} landmarks")
        print(f"   Difíceis: {hard_count} landmarks")
    
    else:
        print("❌ Nenhum dado válido encontrado para gerar relatório")
        
else:
    print("❌ Dados de resumo não disponíveis para gerar relatório")

In [ ]:
print("=== Análise Final e Recomendações ===")

if not results_combined_df.empty:
    # Métricas gerais finais
    total_detections = len(results_combined_df)
    successful_detections = len(results_combined_df.dropna(subset=['Error']))
    overall_success_rate = (successful_detections / total_detections) * 100
    overall_mean_error = results_combined_df['Error'].mean()
    
    print(f"\n📊 Métricas Gerais do Sistema:")
    print(f"   Total de tentativas de detecção: {total_detections:,}")
    print(f"   Detecções bem-sucedidas: {successful_detections:,}")
    print(f"   Taxa de sucesso geral: {overall_success_rate:.1f}%")
    print(f"   Erro médio geral: {overall_mean_error:.3f} mm")
    
    # Avaliação da qualidade
    print(f"\n🎯 Avaliação da Qualidade:")
    if overall_success_rate >= 80:
        quality_detection = "Excelente ✅"
    elif overall_success_rate >= 60:
        quality_detection = "Boa ⚠️"
    else:
        quality_detection = "Precisa melhorar ❌"
    
    if not np.isnan(overall_mean_error):
        if overall_mean_error <= 3.0:
            quality_precision = "Excelente ✅"
        elif overall_mean_error <= 5.0:
            quality_precision = "Boa ⚠️"
        else:
            quality_precision = "Precisa melhorar ❌"
    else:
        quality_precision = "Não avaliável"
    
    print(f"   Taxa de detecção: {quality_detection}")
    print(f"   Precisão: {quality_precision}")
    
    # Recomendações específicas
    print(f"\n💡 Recomendações para Melhoria:")
    
    if overall_success_rate < 80:
        print(f"   🔧 Taxa de detecção baixa - considere:")
        print(f"      • Refinar heurísticas geométricas")
        print(f"      • Treinar modelos ML com mais dados")
        print(f"      • Ajustar parâmetros de confiança")
    
    if not np.isnan(overall_mean_error) and overall_mean_error > 3.0:
        print(f"   🎯 Precisão baixa - considere:")
        print(f"      • Melhorar qualidade da simplificação")
        print(f"      • Adicionar features mais discriminativas no ML")
        print(f"      • Validar orientação e escala das malhas")
    
    # Comparação entre métodos
    methods = results_combined_df['Method'].unique()
    if len(methods) >= 2:
        print(f"\n⚖️  Comparação entre Métodos:")
        for method in methods:
            method_data = results_combined_df[results_combined_df['Method'] == method]
            method_rate = method_data['Detected'].mean() * 100
            method_error = method_data['Error'].mean()
            print(f"   {method}: {method_rate:.1f}% detecção, {method_error:.3f}mm erro")
        
        # Recomendar melhor método
        method_scores = {}
        for method in methods:
            method_data = results_combined_df[results_combined_df['Method'] == method]
            rate = method_data['Detected'].mean() * 100
            error = method_data['Error'].mean()
            # Score combinado (rate ponderado por precisão)
            if not np.isnan(error):
                score = rate * (1 / (1 + error))
            else:
                score = rate * 0.5  # Penalizar falta de dados
            method_scores[method] = score
        
        best_method = max(method_scores, key=method_scores.get)
        print(f"\n🏆 Método Recomendado: {best_method}")
        print(f"   Com base na combinação de taxa de detecção e precisão")

else:
    print("❌ Dados insuficientes para análise final")

print(f"\n✅ Análise completa concluída!")
print(f"📁 Todos os resultados foram salvos em: {RESULTS_DIR}")

In [ ]:
print("=== CONCLUSÕES DA ANÁLISE ===")
print()
print("Esta análise quantitativa demonstrou a capacidade de avaliação sistemática")
print("dos métodos de detecção implementados.")
print()
print("### Principais Contribuições:")
print()
print("1. **Framework de Avaliação Robusto**: Sistema completo para calcular métricas de precisão e robustez")
print("2. **Análise Comparativa**: Comparação objetiva entre métodos geométrico e ML")
print("3. **Visualizações Informativas**: Gráficos que facilitam interpretação dos resultados")
print("4. **Relatórios Automatizados**: Documentação estruturada da performance por landmark")
print("5. **Recomendações Baseadas em Dados**: Sugestões concretas para melhorias")
print()
print("### Aspectos Técnicos Validados:")
print()
print("- ✅ **Métricas Implementadas**: Erro de detecção, MDE, taxa de detecção")
print("- ✅ **Processamento em Lote**: Avaliação automatizada de datasets")
print("- ✅ **Análise Estatística**: Comparações significativas entre métodos")
print("- ✅ **Visualização Profissional**: Gráficos publication-ready")
print("- ✅ **Relatórios Estruturados**: CSV e visualizações para documentação")
print()
print("### Para Trabalhos Futuros:")
print()
print("Este framework de análise está pronto para:")
print("- Avaliação com dados reais (MUG500+, NMDID)")
print("- Comparação com métodos da literatura")
print("- Otimização de hiperparâmetros baseada em métricas")
print("- Validação cruzada e testes estatísticos rigorosos")